In [3]:
import os
import fnmatch
import string
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def get_files(path, pattern='*.conll'):
    
    files = []
    for root, dirnames, filenames in os.walk(path):

        for filename in fnmatch.filter(filenames, pattern):
            files.append(os.path.join(root, filename) )


    # sort the list, to avoid mismatch in the output files
    files = sorted(files)
    
    return files


In [3]:
def read_data(file):
    
    try:
        with open(file,'r') as file:
            lines = [line.strip() for line in file if len(line.strip())!=0]
            
    except:
        print(f'Courrpt File: {file}')
        return []
        
    i = 0
    sentences = []
    
    for j,line in enumerate(lines[1:]):
    
    
        if len(line)>0 and line[0] =='#':
            sentences.append(lines[i:j+1])
            i = j+1
        if len(line)==1:
            print(file)
        else:
            continue
            
    return sentences
                

In [4]:
def get_id_domain(line):
    
    line = line.replace("# id",'')
    ID = line.split('\t')[0].strip()
    domain = line.split('\t')[1].replace('domain=','')

    return ID, domain


def prepare_data(files):
    
    mx = 0
    raw_data = []
    for file in files:
        raw_data.extend(read_data(file))
    
    dataframe = []
    for item in raw_data:
        
        ID, domain = get_id_domain(item[0])
        tokens = []
        label = []
        for entry in item[1:]:
            
            entry = entry.split('_ _')
            tokens.append(entry[0])
            label.append(entry[1])
        
        assert len(label)==len(tokens) 
        
        sens= " ".join(tokens)
        labels= " ".join(label)
        mx = max(mx,len(sens))
        
        dataframe.append([ID,domain,sens,labels])

    
    print(mx)
        
    return dataframe
        

In [5]:
def write_files(dataframe, split):
    
    df = pd.DataFrame(dataframe, columns= ['ID','lang','sent','labels'])
    
    df.to_csv(f'Dataset/{split}.csv',index=False)
    
    return df
    
    
    

In [6]:
    
def create_vocab(sentences):    

    word_string = ' '.join(sentences)
        
    vocab = set(word_string.split())
    return list(vocab)


In [7]:
dataset_path = 'Dataset/'
files = get_files(dataset_path)

train_files = [file for file in files if 'train' in file]
dev_files = [file for file in files if 'dev' in file]


In [8]:
train_data = prepare_data(train_files)
dev_data = prepare_data(dev_files)

328
308


In [46]:
train_df = write_files(train_data, 'train')
dev_df = write_files(dev_data, 'dev')
train_df

,ID,lang,sent,labels
0,7db90724-a10b-4e6c-b95e-355d051a59bf,bn,স্টেশনটি প্ল্যাটফর্ম স্ক্রিন ডোর দিয়ে সজ...,O B-OtherPROD I-OtherPROD I-OtherPROD O O
1,ec02ea49-d32e-4e6f-a634-f0e15c7b8301,bn,উদ্ভিদটির ৯০ ০০০ এরও বেশি সৌর প্যানেল ২...,O O O O O B-OtherPROD I-OtherPROD O O...
2,e6507a5f-a3f9-49ff-a2b8-06add8fb5348,bn,এই মিশনটি চন্দ্র এক্স-রশ্মি মানমন্দির মোত...,O O B-OtherPROD I-OtherPROD I-OtherPROD ...
3,e5779e4e-a2e6-47ad-a9aa-a9b5c3458fbd,bn,সংস্থাটি বেশ কয়েকটি বিমান এবং রাডার স্ক...,O O O O O B-OtherPROD O O O O O O
4,0cef1625-55a9-4e0f-8998-157a06113ff8,bn,ট্রাম স্টেশনটি কোচি কোওচি প্রশাসনিক অঞ্চল...,B-OtherPROD O B-HumanSettlement B-ORG I-O...
...,...,...,...,...
170807,31e243ab-9e06-4d75-9f69-b04bc1253772,zh,肝 性 腦 病 變 中 也 指 示 乳 lactulose .,B-Symptom I-Symptom I-Symptom I-Symptom I...
170808,1fec869b-eab4-4757-b996-dfed74e7ad3c,zh,他 撰 写 了 许 多 关 于 同 性 恋 的 文 章 和 书...,O O O O O O O O O O O O O O O ...
170809,778d1f95-bdcc-48f7-8e45-956a6dc1f6db,zh,为 这 首 歌 支 付 的 钱 捐 赠 给 了 艾 滋 病 研...,O O O O O O O O O O O O B-Symptom...
170810,9acd3b41-3158-4012-9187-cc887b88718f,zh,白 细 胞 是 發 炎 响 应 中 非 常 重 要 的 药 物...,O O O O B-Symptom I-Symptom O O O O ...


In [47]:
tags = create_vocab(train_df['labels'].to_list())
vocab = create_vocab(train_df['sent'].to_list())

In [48]:
with open('vocab.txt','w') as file:
        lines = "\n".join(vocab)
        file.write(lines)
        
with open('tags.txt','w') as file:
        lines = "\n".join(tags)
        file.write(lines)

In [49]:
train_df['sent'].values

array(['স্টেশনটি  প্ল্যাটফর্ম  স্ক্রিন  ডোর  দিয়ে  সজ্জিত। ',
       'উদ্ভিদটির  ৯০  ০০০  এরও  বেশি  সৌর  প্যানেল  ২৩৫  একরও  বেশি  রয়েছে। ',
       'এই  মিশনটি  চন্দ্র  এক্স-রশ্মি  মানমন্দির  মোতায়েন  করেছে। ',
       ..., '为  这  首  歌  支  付  的  钱  捐  赠  给  了  艾  滋  病  研  究  . ',
       '白  细  胞  是  發  炎  响  应  中  非  常  重  要  的  药  物  . ',
       '更  强  烈  的  感  染  可  能  导  致  严  重  的  發  炎  周  围  感  染  的  肌  肉  纤  维  . '],
      dtype=object)

In [52]:
df = train_df[train_df['lang']=='bn']
df

,ID,lang,sent,labels
0,7db90724-a10b-4e6c-b95e-355d051a59bf,bn,স্টেশনটি প্ল্যাটফর্ম স্ক্রিন ডোর দিয়ে সজ...,O B-OtherPROD I-OtherPROD I-OtherPROD O O
1,ec02ea49-d32e-4e6f-a634-f0e15c7b8301,bn,উদ্ভিদটির ৯০ ০০০ এরও বেশি সৌর প্যানেল ২...,O O O O O B-OtherPROD I-OtherPROD O O...
2,e6507a5f-a3f9-49ff-a2b8-06add8fb5348,bn,এই মিশনটি চন্দ্র এক্স-রশ্মি মানমন্দির মোত...,O O B-OtherPROD I-OtherPROD I-OtherPROD ...
3,e5779e4e-a2e6-47ad-a9aa-a9b5c3458fbd,bn,সংস্থাটি বেশ কয়েকটি বিমান এবং রাডার স্ক...,O O O O O B-OtherPROD O O O O O O
4,0cef1625-55a9-4e0f-8998-157a06113ff8,bn,ট্রাম স্টেশনটি কোচি কোওচি প্রশাসনিক অঞ্চল...,B-OtherPROD O B-HumanSettlement B-ORG I-O...
...,...,...,...,...
9702,5a7ba7a2-4433-4c0e-86fa-f5923e8f3610,bn,এটি সুরিন্দর ফিল্মস দ্বারা উত্পাদিত হয়।,O B-PrivateCorp I-PrivateCorp O O O
9703,dc324b57-e06a-421d-b38b-38573df569f1,bn,অংশীদারের মৃত্যুর পরে অংশীদারি কারবার এর ...,O O O B-PrivateCorp I-PrivateCorp O O ...
9704,efe243f1-95eb-4f72-a4fe-56a9380e02d6,bn,এটি ১৯৮০ এর দশকে বেল ল্যাবস্ এ তৈরি কর...,O O O O B-PrivateCorp I-PrivateCorp O ...
9705,37c3d11f-ed92-4304-a292-8186d7e3ea3c,bn,১৯৫৭ সালে তিনি বেল ল্যাবস্ এ গণিতবিদ হি...,O O O B-PrivateCorp I-PrivateCorp O O ...


In [6]:
SEED = 42
LANG = 'bn'
df = pd.read_csv('./Dataset/train.csv')
train_df, dev_df = train_test_split(df, test_size=0.2, random_state=SEED)


if LANG!=None:
    train_df = train_df[train_df['lang']==LANG]
    dev_df = dev_df[dev_df['lang']==LANG]

In [7]:
train_df

,ID,lang,sent,labels
2315,59e1a4c5-ece6-4677-b781-689981533d44,bn,ইহুদিরা হিজরত বইটিতে তাওরাত বইতে বর্ণিত ...,O B-WrittenWork O B-WrittenWork O O O ...
6231,c98f2083-b94d-48f7-8347-d7eb70f6ee90,bn,মূলত টাইফাস জ্বর কলেরা এবং আমাশয় এর কা...,O B-Disease I-Disease B-Disease O B-Symp...
4515,c0067b79-3480-45e6-a557-de1430fcd615,bn,সেরা একক : নেভার গনা গিভ ইউ আপ রিক অ্...,O O O B-MusicalWork I-MusicalWork I-Musi...
2212,ab3bd4a3-09ea-4ed6-913d-44a3ad03a18e,bn,প্রথম পোপ গ্রেগরি ( ডি। ৬০৪ ) ইয়োব এর...,B-Cleric I-Cleric I-Cleric O O O O B-W...
9367,3697711a-a221-45d1-af4c-f3ec7828672b,bn,এই গাছের ফুলগুলি স্তন্যপায়ী প্রাণী এবং ...,O O O O O O O O O O O O O O O ...
...,...,...,...,...
8792,307f4c17-8729-4079-b6c9-acd7922be309,bn,গানটি একটি যুবক স্বর্ণকেশী মেয়ে সম্পর্কে...,O O O O O O O O O O B-ArtWork I-Ar...
3890,7274e80c-9495-4368-8a60-3d685e20718e,bn,২১ ডিসেম্বর স্নো হোয়াইট অ্যান্ড দ্য সেভ...,O O B-VisualWork I-VisualWork I-VisualWor...
2747,d78ed44d-5a63-42dd-8f54-1f5def0896f5,bn,কমপ্যাক্ট ডিস্ক ( সিডি ) যৌথভাবে সনি এব...,O O O O O O B-PublicCorp O B-PublicCo...
5311,e35e5ccf-c919-421b-a10a-7e39a456c475,bn,রেস্তোঁরাটিকে তাজা গুল্ম সরবরাহ করতে হোটে...,O O O O O O O B-Facility O O


In [14]:
def recreate_conll_format(data):
    id = data['ID']
    lang = data['lang']
    lines = [f'# id {id} \t domain={lang}']
    sents = data['sent'].split()
    labels = data['labels'].split()
    for token,label in zip(sents,labels):
        line = f'{token} _ _ {label}'
        lines.append(line)
    
    return "\n".join(lines)

In [20]:
fileName = f'{LANG}-train.conll'
with open(fileName,'w') as file:
    for ind in range(train_df.shape[0]):
        data = train_df.iloc[ind]
        lines = recreate_conll_format(data)
        file.write("\n"+lines+"\n\n")

fileName = f'{LANG}-dev.conll'
with open(fileName,'w') as file:
    for ind in range(dev_df.shape[0]):
        data = dev_df.iloc[ind]
        lines = recreate_conll_format(data)
        file.write("\n"+lines+"\n\n")